In [1]:
# Program for learning the performance of 2 layer deep neural network pyramid strucuture

import numpy
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras.datasets import cifar10
from matplotlib import pyplot 
from scipy.misc import toimage
import datetime
from keras.layers import Activation, Dense

K.set_image_dim_ordering('th')

Using TensorFlow backend.


In [2]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

#dataset image parameters
img_rows=32
img_cols=32
channel=3
nb_classes=10

In [3]:
(x_train, y_train),(x_test,y_test)=cifar10.load_data()

x_train = x_train.reshape(x_train.shape[0], channel,img_rows, img_cols)
x_test = x_test.reshape(x_test.shape[0], channel,img_rows, img_cols)
input_shape=(3,img_rows,img_cols)

In [4]:
x_test=x_test.astype('float32')
x_train=x_train.astype('float32')
x_test=x_test/255.
x_train=x_train/255.

In [5]:
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]


In [6]:
# Defining model
vgg = Sequential()
vgg.add(Convolution2D(30, 2, 2, input_shape=input_shape, border_mode='valid'))
vgg.add(Activation('relu'))
#vgg.add(MaxPooling2D((2, 2), strides=(2, 2), border_mode='valid'))

vgg.add(Convolution2D(40, 2, 2, border_mode='valid'))
vgg.add(Activation('relu'))
vgg.add(MaxPooling2D((2, 2), strides=(2, 2), border_mode='valid'))

vgg.add(Flatten())
vgg.add(Dense(200))
vgg.add(Dropout(0.5))
vgg.add(Activation('relu'))
vgg.add(Dense(nb_classes, activation='softmax'))

model = vgg
print (model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 30, 31, 31)    390         convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 30, 31, 31)    0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 40, 30, 30)    4840        activation_1[0][0]               
____________________________________________________________________________________________________
activation_2 (Activation)        (None, 40, 30, 30)    0           convolution2d_2[0][0]            
___________________________________________________________________________________________

In [7]:
# Compile model
epochs = 25
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 30, 31, 31)    390         convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 30, 31, 31)    0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 40, 30, 30)    4840        activation_1[0][0]               
____________________________________________________________________________________________________
activation_2 (Activation)        (None, 40, 30, 30)    0           convolution2d_2[0][0]            
___________________________________________________________________________________________

In [8]:
# printing the time when the training starts
print(datetime.datetime.now())

# Fit the model
model.fit(x_train, y_train, validation_split=0.2, nb_epoch=epochs, batch_size=32)

# printing the time when the training finishes
print(datetime.datetime.now())


2017-04-21 08:10:39.742392
Train on 40000 samples, validate on 10000 samples
Epoch 1/25
40000/40000 [==============================] - 76s - loss: 1.7591 - acc: 0.3616 - val_loss: 1.4122 - val_acc: 0.4904
Epoch 2/25
40000/40000 [==============================] - 145s - loss: 1.3700 - acc: 0.5072 - val_loss: 1.2209 - val_acc: 0.5688
Epoch 3/25
40000/40000 [==============================] - 173s - loss: 1.1922 - acc: 0.5780 - val_loss: 1.1088 - val_acc: 0.6071
Epoch 4/25
40000/40000 [==============================] - 173s - loss: 1.0717 - acc: 0.6189 - val_loss: 1.0463 - val_acc: 0.6321
Epoch 5/25
40000/40000 [==============================] - 173s - loss: 0.9743 - acc: 0.6530 - val_loss: 0.9731 - val_acc: 0.6573
Epoch 6/25
40000/40000 [==============================] - 174s - loss: 0.9031 - acc: 0.6795 - val_loss: 0.9492 - val_acc: 0.6661
Epoch 7/25
40000/40000 [==============================] - 173s - loss: 0.8254 - acc: 0.7067 - val_loss: 0.9454 - val_acc: 0.6687
Epoch 8/25
40000/4000

In [9]:
# Final evaluation of the model
scores = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 69.01%
